In [ ]:
# =========================================
# MOUNT DRIVE
# =========================================
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# =========================================
# PATHS
# =========================================
BASE_PATH = "/content/drive/MyDrive/Chatbot"
IMG_DIR = BASE_PATH + "/DeepWeeds/images/"
LABEL_DIR = BASE_PATH + "/DeepWeeds/labels/"
MODEL_SAVE_DIR = BASE_PATH + "/"

BATCH_SIZE = 32
IMG_SIZE = (224, 224)
INPUT_SHAPE = (224, 224, 3)

CLASSES = [0,1,2,3,4,5,6,7,8]
CLASS_STR = [str(i) for i in CLASSES]

# =========================================
# LOAD CSV
# =========================================
train_df = pd.read_csv(LABEL_DIR + "train_subset0.csv")
val_df   = pd.read_csv(LABEL_DIR + "val_subset0.csv")

train_df["Label"] = train_df["Label"].astype(str)
val_df["Label"] = val_df["Label"].astype(str)

# =========================================
# DATA GENERATORS (IMPORTANT FIX)
# =========================================
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
).flow_from_dataframe(
    train_df,
    IMG_DIR,
    x_col="Filename",
    y_col="Label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    classes=CLASS_STR,
    shuffle=True
)

val_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
).flow_from_dataframe(
    val_df,
    IMG_DIR,
    x_col="Filename",
    y_col="Label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    classes=CLASS_STR,
    shuffle=False
)

# =========================================
# OPTION 1: START FRESH MODEL
# =========================================
def build_model():
    base_model = ResNet50(
        weights="imagenet",
        include_top=False,
        input_shape=INPUT_SHAPE
    )

    # Freeze initially
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    output = Dense(len(CLASSES), activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=output)

    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model, base_model


# =========================================
# OPTION 2: LOAD PRETRAINED MODEL
# =========================================
LOAD_PRETRAINED = False
PRETRAINED_PATH = BASE_PATH + "/resnet_epoch_39.h5"

if LOAD_PRETRAINED and os.path.exists(PRETRAINED_PATH):
    print("Loading pretrained model...")
    model = load_model(PRETRAINED_PATH)
    base_model = model.layers[0]  # ResNet base
else:
    model, base_model = build_model()

# =========================================
# CALLBACKS
# =========================================
checkpoint = ModelCheckpoint(
    filepath=MODEL_SAVE_DIR + "resnet_epoch_{epoch:02d}.h5",
    save_freq="epoch",
    save_best_only=False
)

early_stop = EarlyStopping(
    monitor="val_accuracy",
    patience=5,
    restore_best_weights=True
)

# =========================================
# PHASE 1 — TRAIN HEAD ONLY
# =========================================
print("\nStarting Phase 1 (Frozen Base)...\n")

model.fit(
    train_gen,
    epochs=5,
    validation_data=val_gen,
    callbacks=[checkpoint, early_stop]
)

# =========================================
# PHASE 2 — FINE TUNE LAST 50 LAYERS
# =========================================
print("\nUnfreezing last 50 layers...\n")

for layer in base_model.layers[-50:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.fit(
    train_gen,
    epochs=20,
    validation_data=val_gen,
    callbacks=[checkpoint, early_stop]
)

# =========================================
# SAVE FINAL MODEL
# =========================================
model.save(MODEL_SAVE_DIR + "resnet_final_finetuned.h5")
print("Training complete.")


Mounted at /content/drive
Found 10501 validated image filenames belonging to 9 classes.
Found 3501 validated image filenames belonging to 9 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step

Starting Phase 1 (Frozen Base)...



/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 24s/step - accuracy: 0.4748 - loss: 1.6736 

329/329 ━━━━━━━━━━━━━━━━━━━━ 10431s 32s/step - accuracy: 0.4750 - loss: 1.6728 - val_accuracy: 0.6252 - val_loss: 1.0810
Epoch 2/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.6456 - loss: 1.0059

329/329 ━━━━━━━━━━━━━━━━━━━━ 77s 233ms/step - accuracy: 0.6457 - loss: 1.0058 - val_accuracy: 0.6978 - val_loss: 0.8777
Epoch 3/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.6978 - loss: 0.8484

...

329/329 ━━━━━━━━━━━━━━━━━━━━ 83s 251ms/step - accuracy: 0.9992 - loss: 0.0069 - val_accuracy: 0.8783 - val_loss: 0.4361
Epoch 15/20
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step - accuracy: 0.9997 - loss: 0.0042

329/329 ━━━━━━━━━━━━━━━━━━━━ 88s 262ms/step - accuracy: 0.9997 - loss: 0.0042 - val_accuracy: 0.8769 - val_loss: 0.4473
Epoch 16/20
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 1.0000 - loss: 0.0034

329/329 ━━━━━━━━━━━━━━━━━━━━ 81s 247ms/step - accuracy: 1.0000 - loss: 0.0034 - val_accuracy: 0.8763 - val_loss: 0.4415
Epoch 17/20
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - accuracy: 0.9997 - loss: 0.0040

329/329 ━━━━━━━━━━━━━━━━━━━━ 85s 256ms/step - accuracy: 0.9997 - loss: 0.0040 - val_accuracy: 0.8783 - val_loss: 0.4515


Training complete.
